# Imports

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import math

2024-06-05 14:32:05.373447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 14:32:05.373602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 14:32:05.518327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Read Data

In [2]:
X_train = pd.read_csv("/kaggle/input/ibm-classification-feature-selection/X_train_normalized.csv", index_col=[0])
X_test = pd.read_csv("/kaggle/input/ibm-classification-feature-selection/X_test_normalized.csv", index_col=[0])
y_train = pd.read_csv("/kaggle/input/ibm-classification-feature-selection/y_train.csv", index_col=[0])
y_test = pd.read_csv("/kaggle/input/ibm-classification-feature-selection/y_test.csv", index_col=[0]).reset_index(drop=True)

In [3]:
X_train.head(5)

,Symbol,Volume,Sector_Basic Materials,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,...,Adj Close 14-Day std Differenced,Adj Close 26-Day std Differenced,Adj Close 7-Day Upper Bollinger Band Differenced,Distance to Adj Close 7-Day Upper Bollinger Band Differenced,True Range Differenced,True Range 26-Day SMA Differenced,Adj Close Differenced,High Differenced,Low Differenced,Open Differenced
0,-1.439324,-0.326871,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,0.041515,0.037984,-0.002604,-0.016884,-0.066039,0.104504,-0.020330,0.008943,0.015877,0.062622
1,-1.069061,-0.344648,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,2.582087,-0.412467,...,-0.410893,-0.015098,0.023690,-0.111938,-0.141182,-0.090630,-0.107768,0.032258,0.030478,0.096788
2,1.641263,-0.295005,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,0.081853,0.002250,0.188618,0.000862,-0.002066,0.105746,0.125379,0.102205,0.158246,0.137784
3,-0.343346,0.285874,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,2.582087,-0.412467,...,0.162828,-0.007191,0.102354,0.062595,0.043600,0.091515,0.136523,0.094434,0.004925,0.076288
4,1.256190,-0.328472,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,2.534516,-0.387284,-0.412467,...,1.195841,-0.202535,1.265493,-0.284640,-0.679202,-1.099516,0.521058,0.755054,0.866425,0.817676


In [4]:
X_test.head(5)

,Symbol,Volume,Sector_Basic Materials,Sector_Communication Services,Sector_Consumer Cyclical,Sector_Consumer Defensive,Sector_Energy,Sector_Financial Services,Sector_Healthcare,Sector_Industrials,...,Adj Close 14-Day std Differenced,Adj Close 26-Day std Differenced,Adj Close 7-Day Upper Bollinger Band Differenced,Distance to Adj Close 7-Day Upper Bollinger Band Differenced,True Range Differenced,True Range 26-Day SMA Differenced,Adj Close Differenced,High Differenced,Low Differenced,Open Differenced
0,-1.054250,-0.283809,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,2.424439,...,0.244619,-0.054833,0.193442,-0.988178,-0.460024,-0.458573,-0.961706,-0.123181,-0.779922,0.216366
1,0.449017,-0.359859,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,2.534516,-0.387284,-0.412467,...,-1.623606,-1.598235,-0.116220,-0.549677,-0.673831,-0.614296,-0.682605,0.832754,-0.079040,1.432648
2,1.419105,-0.198809,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,-4.366291,-2.195166,0.093043,-4.112427,3.947181,1.902798,-4.471950,1.435083,-4.328154,1.015831
3,-1.261598,-0.272044,-0.21136,-0.21136,-0.356345,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,5.267571,4.461425,4.610485,-12.390394,14.154228,10.022441,-10.617055,-3.111471,-14.261016,-12.872375
4,-0.128593,-0.255868,-0.21136,-0.21136,2.806268,-0.279992,-0.222173,-0.394553,-0.387284,-0.412467,...,0.039264,0.059860,0.030323,-0.411204,0.082851,0.121352,-0.433286,-0.263072,-0.294412,0.025041


In [5]:
y_train.head(5)

,1-week Forward Return Sign
0,0.0
1,0.0
2,0.0
3,0.0
4,1.0


In [6]:
y_test.head(5)

,1-week Forward Return Sign
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


# Data Prep

LSTMs requires the data to be structured differently to most ML models. The idea is to look at a 'window' of data (e.g. last 5 timesteps) and use it to predict the target at the next timestep. We can roll the window to look at the whole dataset but only looking at one window at a time. 

To do this, we convert our dataframe to a 3D array X and a 1D array y. The array X has shape (n_samples, window_size, num_features) and the array y simply contains the target for each sample. The structure of the array X is shown below for a window size of 2 (LHS) and the array y is shown on the RHS.

Note that when we split into our training and validation sets, we split by time rather than by stock to avoid look-ahead bias.

[Stock1Feature1(1), Stock1Feature2(1), ....., Stock1Featuren(1)], [Stock1Feature1(0), Stock1Feature2(0), ....., Stock1Featuren(0)]                                                                             [Stock1Target(2)]

[Stock1Feature1(2), Stock1Feature2(2), ....., Stock1Featuren(2)], [Stock1Feature1(1), Stock1Feature2(1), ....., Stock1Featuren(1)]                                                                             [Stock1Target(3)]

[Stock1Feature1(3), Stock1Feature2(3), ....., Stock1Featuren(3)], [Stock1Feature1(2), Stock1Feature2(2), ....., Stock1Featuren(2)]                                                                             [Stock1Target(4)]

....





In [7]:
# Append targets to dataframe for ease
df = pd.concat([X_train, y_train], axis=1)

# Get the first 1000 occurrences of each symbol for training and the remaining for validation (equivalent to splitting by time but we don't have a time column)
train = df.groupby('Symbol').head(1000)
val = df.drop(train.index)

# Reset indices
train.reset_index(inplace=True, drop=True)
val.reset_index(inplace=True, drop=True)
train.shape, val.shape

((467943, 123), (164384, 123))

In [8]:
def prep_data_for_lstm(df, window):
    """function to put data in the correct format for LSTM. It assumes that the 'Target' is the last column of the dataframe."""
    
    # Get unique stock ids
    stock_ids = df['Symbol'].unique()

    # Array to store X and y
    X = []
    y = []
    
    for s in stock_ids:
        stock_data = df[df['Symbol'] == s].to_numpy()

        # Loop through each index in the dataframe and get X and y
        for i in range(len(stock_data)-window):
            # Get values in window and wrap each one in a list
            row = stock_data[i: i+window]
            X.append(row)

            # Get the label (first value after the window)
            label = stock_data[i + window][-1]
            y.append(label)

    return np.array(X), np.array(y)

In [9]:
# Window size - use last n samples to predict next sample
window = 5

# Prep training data for lstm
X_train_lstm, y_train_lstm = prep_data_for_lstm(
    df=train,
    window=window
)

# Print val data for lstm
X_val_lstm, y_val_lstm = prep_data_for_lstm(
    df=val,
    window=window
)
print(X_train_lstm)
print(y_train_lstm)
print(X_train_lstm.shape, X_val_lstm.shape)
print(y_train_lstm.shape, y_val_lstm.shape)

[[[-1.43932383e+00 -3.26870525e-01 -2.11359937e-01 ...  1.58770652e-02
    6.26215242e-02  0.00000000e+00]
  [-1.43932383e+00 -2.73341572e-01 -2.11359937e-01 ... -3.15791316e-02
   -5.69572700e-02  0.00000000e+00]
  [-1.43932383e+00 -2.91855060e-01 -2.11359937e-01 ...  7.06343224e-02
    8.31211929e-02  0.00000000e+00]
  [-1.43932383e+00 -3.28487324e-01 -2.11359937e-01 ... -1.26490829e-01
   -5.35406585e-02  1.00000000e+00]
  [-1.43932383e+00 -3.26293639e-01 -2.11359937e-01 ... -2.06275409e-02
   -5.01240471e-02  1.00000000e+00]]

 [[-1.43932383e+00 -2.73341572e-01 -2.11359937e-01 ... -3.15791316e-02
   -5.69572700e-02  0.00000000e+00]
  [-1.43932383e+00 -2.91855060e-01 -2.11359937e-01 ...  7.06343224e-02
    8.31211929e-02  0.00000000e+00]
  [-1.43932383e+00 -3.28487324e-01 -2.11359937e-01 ... -1.26490829e-01
   -5.35406585e-02  1.00000000e+00]
  [-1.43932383e+00 -3.26293639e-01 -2.11359937e-01 ... -2.06275409e-02
   -5.01240471e-02  1.00000000e+00]
  [-1.43932383e+00 -3.31766466e-01 

# Fit LSTM
An explanation of the model's layers can be seen in the comments in the code.

In [10]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.regularizers import l2

In [11]:
# Initialize sequential model in keras. This is a model that expects a linear stack of layers, meaning that each layer has one input tensor and one output tensor. 
model = Sequential()

# Add input layer. Each input tensor will be an element of X_train_lstm so the tensor will have shape (window, n_features)
model.add(InputLayer((window, X_train_lstm.shape[2]))) # Window size and number of variables used to predict

# Add LSTM layer. This is a recurrent layer that can learn long-term dependencies in sequences.
model.add(
    LSTM(
        units=64,
        dropout=0.2, # Randomly set fraction of non-recurrent LSTM units to 0 at each training time step to reduce relianc on units and prevent overfitting
        recurrent_dropout=0.2 # Randomly set fraction of recurrent LSTM units to 0 at each training time step to reduce relianc on units and prevent overfitting
    )
)

# Add hidden layer to take LSTM layer output and transform it to a higher-level feayire space.
model.add(
    Dense(
        units=8, 
        activation='relu',  # relu activation function adds some non-linearity without being computatinally expensive
        kernel_regularizer=l2(0.01) # Add l2 regularization on layer to reduce magnitude of weights reducing overfitting
    )
)

# Add output layer with sigmoid activation function to output a value between 0 and 1 interpreted as probability of postiivre class
model.add(Dense(1, 'sigmoid'))

# Print summary of model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        48,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,657 (190.07 KB)

 Trainable params: 48,657 (190.07 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Checpoint for saving model
cp1 = ModelCheckpoint(
    'lstm_model.keras',
    save_best_only=True
)

# Early stopping rounds checkpint
cp2 =  EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [13]:
# Compile model
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.01),
    metrics=[AUC(name='auc')]
)

In [14]:
# Adjust class weights inversely proportional to class frequencies
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(y_train_lstm),
    y=y_train_lstm
)
class_weight_dict = dict(enumerate(class_weights))

# Fit model by measiromg AUC against validation set.
model.fit(
    X_train_lstm,
    y_train_lstm,
    validation_data=(X_val_lstm, y_val_lstm),
    epochs=100, # Number of times to run through the data
    callbacks=[cp1, cp2], # At end of every epoch, save the model if loss was lower than before. Also EarlyStoppingRounds
    class_weight=class_weight_dict # Handle imbalance of classes
)

Epoch 1/100
14551/14551 ━━━━━━━━━━━━━━━━━━━━ 82s 5ms/step - auc: 0.7608 - loss: 0.5920 - val_auc: 0.7713 - val_loss: 0.5798
Epoch 2/100
14551/14551 ━━━━━━━━━━━━━━━━━━━━ 78s 5ms/step - auc: 0.7884 - loss: 0.5655 - val_auc: 0.7995 - val_loss: 0.5456
Epoch 3/100
14551/14551 ━━━━━━━━━━━━━━━━━━━━ 78s 5ms/step - auc: 0.7894 - loss: 0.5622 - val_auc: 0.8059 - val_loss: 0.5383
Epoch 4/100
14551/14551 ━━━━━━━━━━━━━━━━━━━━ 81s 6ms/step - auc: 0.7900 - loss: 0.5622 - val_auc: 0.7962 - val_loss: 0.5548
Epoch 5/100
14551/14551 ━━━━━━━━━━━━━━━━━━━━ 80s 6ms/step - auc: 0.7914 - loss: 0.5613 - val_auc: 0.8077 - val_loss: 0.5319
Epoch 6/100
14551/14551 ━━━━━━━━━━━━━━━━━━━━ 80s 5ms/step - auc: 0.7944 - loss: 0.5578 - val_auc: 0.8056 - val_loss: 0.5366
Epoch 7/100
14551/14551 ━━━━━━━━━━━━━━━━━━━━ 79s 5ms/step - auc: 0.7949 - loss: 0.5572 - val_auc: 0.7995 - val_loss: 0.5540
Epoch 8/100
14551/14551 ━━━━━━━━━━━━━━━━━━━━ 79s 5ms/step - auc: 0.7947 - loss: 0.5575 - val_auc: 0.7973 - val_loss: 0.5474
Epoch 9/